# Quick Start Guide to GenAIRR

Welcome to the Quick Start Guide for GenAIRR, a Python module designed for generating synthetic Adaptive Immune Receptor Repertoire (AIRR) sequences. This guide will walk you through the basic usage of GenAIRR, including setting up your environment, simulating heavy and light chain sequences, and customizing your simulations.


## Installation

Before you begin, ensure that you have Python 3.x installed on your system. GenAIRR can be installed using pip, Python's package installer. Execute the following command in your terminal:


In [ ]:
import pandas as pd
# Install GenAIRR using pip
!pip install GenAIRR

## Setting Up

To start using GenAIRR, you need to import the necessary classes from the module. We'll also set up a `DataConfig` object to specify our configuration.


In [ ]:
# Importing GenAIRR classes
from GenAIRR.simulation import HeavyChainSequenceAugmentor, LightChainSequenceAugmentor, SequenceAugmentorArguments
from GenAIRR.utilities import DataConfig
from GenAIRR.data import builtin_heavy_chain_data_config,builtin_kappa_chain_data_config,builtin_lambda_chain_data_config
# Initialize DataConfig with the path to your configuration
data_config = DataConfig('/path/to/your/config')
# Or Use one of Our Builtin Data Configs
data_config_builtin = builtin_heavy_chain_data_config()


# Set up augmentation arguments (if you have specific requirements)
args = SequenceAugmentorArguments()


## Simulating Heavy Chain Sequences

Let's simulate a heavy chain sequence using `HeavyChainSequenceAugmentor`. This example demonstrates a simple simulation with default settings.


In [ ]:
# Initialize the HeavyChainSequenceAugmentor
heavy_augmentor = HeavyChainSequenceAugmentor(data_config_builtin, args)

# Simulate a heavy chain sequence
heavy_sequence = heavy_augmentor.simulate_augmented_sequence

# Print the simulated heavy chain sequence
print("Simulated Heavy Chain Sequence:", heavy_sequence)


## Customizing Simulations

GenAIRR allows for extensive customization to closely mimic the natural diversity of immune sequences. Below is an example of how to customize mutation rates and indel simulations.


In [ ]:
# Customize augmentation arguments
custom_args = SequenceAugmentorArguments(min_mutation_rate=0.01, max_mutation_rate=0.05, simulate_indels=True, max_indels=3,
                                         corrupt_proba=0.7,save_ns_record=True,save_mutations_record=True)

# Use custom arguments to simulate a heavy chain sequence
custom_heavy_augmentor = HeavyChainSequenceAugmentor(data_config_builtin, custom_args)
custom_heavy_sequence = custom_heavy_augmentor.simulate_augmented_sequence()

# Print the customized heavy chain sequence
print("Customized Simulated Heavy Chain Sequence:", custom_heavy_sequence)


## Generating Naïve Sequences

In immunogenetics, a naïve sequence refers to an antibody sequence that has not undergone the process of somatic hypermutation. GenAIRR allows you to simulate such naïve sequences using the `HeavyChainSequence` class. Let's start by generating a naïve heavy chain sequence.


In [ ]:
from GenAIRR.sequence import HeavyChainSequence

# Create a naive heavy chain sequence
naive_heavy_sequence = HeavyChainSequence.create_random(data_config_builtin)

# Access the generated naive sequence
naive_sequence = naive_heavy_sequence

print("Naïve Heavy Chain Sequence:", naive_sequence)
print('Ungapped Sequence: ')
print(naive_sequence.ungapped_seq)


## Applying Mutations

To mimic the natural diversity and evolution of immune sequences, GenAIRR supports the simulation of mutations through various models. Here, we demonstrate how to apply mutations to a naïve sequence using the `S5F` and `Uniform` mutation models from the mutations submodule.


### Using the S5F Mutation Model

The `S5F` model is a sophisticated mutation model that considers context-dependent mutation probabilities. It's particularly useful for simulating realistic somatic hypermutations.


In [ ]:
from GenAIRR.mutation import S5F

# Initialize the S5F mutation model with custom mutation rates
s5f_model = S5F(min_mutation_rate=0.01, max_mutation_rate=0.05)

# Apply mutations to the naive sequence using the S5F model
s5f_mutated_sequence, mutations, mutation_rate = s5f_model.apply_mutation(naive_heavy_sequence)

print("S5F Mutated Heavy Chain Sequence:", s5f_mutated_sequence)
print("S5F Mutation Details:", mutations)
print("S5F Mutation Rate:", mutation_rate)


### Using the Uniform Mutation Model

The `Uniform` mutation model applies mutations at a uniform rate across the sequence, providing a simpler alternative to the context-dependent models.


In [ ]:
from GenAIRR.mutation import Uniform

# Initialize the Uniform mutation model with custom mutation rates
uniform_model = Uniform(min_mutation_rate=0.01, max_mutation_rate=0.05)

# Apply mutations to the naive sequence using the Uniform model
uniform_mutated_sequence, mutations, mutation_rate = uniform_model.apply_mutation(naive_heavy_sequence)

print("Uniform Mutated Heavy Chain Sequence:", uniform_mutated_sequence)
print("Uniform Mutation Details:", mutations)
print("Uniform Mutation Rate:", mutation_rate)


## Common Use Cases

GenAIRR is a versatile tool designed to meet a broad range of needs in immunogenetics research. This section provides examples and explanations for some common use cases, including generating multiple sequences, simulating specific allele combinations, and more.


### Generating Many Sequences

One common requirement is to generate a large dataset of synthetic AIRR sequences for analysis or benchmarking. Below is an example of how to generate multiple sequences using GenAIRR in a loop.


In [ ]:
num_sequences = 5  # Number of sequences to generate

heavy_sequences = []
for _ in range(num_sequences):
    # Simulate a heavy chain sequence
    heavy_sequence = heavy_augmentor.simulate_augmented_sequence()
    heavy_sequences.append(heavy_sequence)

# Display the generated sequences
for i, seq in enumerate(heavy_sequences, start=1):
    print(f"Heavy Chain Sequence {i}: {seq}")


In [ ]:
import pandas as pd
pd.DataFrame(heavy_sequences)

### Generating a Specific Allele Combination Sequence

In some cases, you might want to simulate sequences with specific V, D, and J allele combinations. Here's how to specify alleles for your simulations.


In [ ]:
# Define your specific alleles
v_allele = 'IGHVF6-G21*01'
d_allele = 'IGHD5-18*01'
j_allele = 'IGHJ6*03'

# Extract the allele objects from data_config
v_allele = next((allele for family in data_config_builtin.v_alleles.values() for allele in family if allele.name == v_allele), None)
d_allele = next((allele for family in data_config_builtin.d_alleles.values() for allele in family if allele.name == d_allele), None)
j_allele = next((allele for family in data_config_builtin.j_alleles.values() for allele in family if allele.name == j_allele), None)

# Check if all alleles were found
if not v_allele or not d_allele or not j_allele:
    raise ValueError("One or more specified alleles could not be found in the data config.")


# Generate a sequence with the specified allele combination
specific_allele_sequence = HeavyChainSequence([v_allele, d_allele, j_allele], data_config_builtin)
specific_allele_sequence.mutate(s5f_model)



print("Specific Allele Combination Sequence:", specific_allele_sequence.mutated_seq)


### Simulating Sequences with Custom Mutation Rates

Adjusting mutation rates allows for the simulation of sequences at various stages of affinity maturation. Here's how to customize mutation rates in your simulations.


In [ ]:
# Customize augmentation arguments with your desired mutation rates
custom_args = SequenceAugmentorArguments(min_mutation_rate=0.15, max_mutation_rate=0.3)

# Initialize the augmentor with custom arguments
custom_augmentor = HeavyChainSequenceAugmentor(data_config_builtin, custom_args)

# Generate a sequence with the custom mutation rates
custom_mutation_sequence = custom_augmentor.simulate_augmented_sequence()

print("Custom Mutation Rate Sequence:", custom_mutation_sequence)


### Generating Naïve vs. Mutated Sequence Pairs

Comparing naïve and mutated versions of the same sequence can be useful for studying somatic hypermutation effects. Here's how to generate such pairs with GenAIRR.


In [ ]:
# Generate a naive sequence
sequence_object = HeavyChainSequence.create_random(data_config_builtin)
sequence_object.mutate(s5f_model)

print("Naïve Sequence:", sequence_object.ungapped_seq)
print("Mutated Sequence:", sequence_object.mutated_seq)


## Conclusion

This section highlighted some common use cases for GenAIRR, demonstrating its flexibility in simulating AIRR sequences for various research purposes. Whether you need large datasets, specific allele combinations, custom mutation rates, or comparative analyses of naïve and mutated sequences, GenAIRR provides the necessary tools to achieve your objectives.
